### Проверка значимости отличий средней выручки с пользователя на неделе перед экспериментом (с 2022.03.16 по 2022.03.23)

In [ ]:
# Возьмём те же группы, что и в эксперименте с изменением дизайна сайта, 
# и проверим значимость отличий средней выручки с пользователя на неделе перед экспериментом (c 2022.03.16 по 2022.03.23).
# Для решения используются данные из файлов 2022-04-01T12_df_sales.csv и experiment_users.csv.

# Importing data
import os
import pandas as pd
from datetime import datetime
from scipy import stats

URL_BASE = 'https://raw.githubusercontent.com/ab-courses/simulator-ab-datasets/main/2022-04-01/'

def read_database(file_name):
    return pd.read_csv(os.path.join(URL_BASE, file_name))

df_sales = read_database('2022-04-01T12_df_sales.csv')
df_experiment = read_database('experiment_users.csv')

print(f"df_sales: \n {df_sales.head(3)}")
print(f"df_experiment: \n {df_experiment.head(3)}")
df_sales['date'] = pd.to_datetime(df_sales['date'])

begin_date = datetime(2022, 3, 16)
end_date = datetime(2022, 3, 23)

df_sales: 
    sale_id                 date  count_pizza  count_drink  price user_id
0  1000001  2022-02-04 10:00:24            1            0    720  1c1543
1  1000002  2022-02-04 10:02:28            1            1    930  a9a6e8
2  1000003  2022-02-04 10:02:35            3            1   1980  23420a
df_experiment: 
   user_id  pilot
0  0ffc65      0
1  b962b9      0
2  7ea63f      0


In [24]:
df_revenue = (
    df_sales[(df_sales['date'] >= begin_date) & (df_sales['date'] < end_date)]
    .groupby('user_id')[['price']].sum()
    .reset_index()
)
df = pd.merge(df_experiment, df_revenue, on='user_id', how='left').fillna(0)

df_control = df[df['pilot'] == 0]['price']
df_pilot = df[df['pilot'] == 1]['price']
t, p_value = stats.ttest_ind(df_control, df_pilot)
print(f"p-value равно: {p_value:.3f}, H0 {'отклоняем' if p_value<0.05 else 'не можем отклонить'}")

      user_id  pilot   price
0      0ffc65      0     0.0
1      b962b9      0     0.0
2      7ea63f      0     0.0
3      7f9a61      0     0.0
4      459e55      0  2160.0
...       ...    ...     ...
23328  b10ce5      1     0.0
23329  c5486d      1     0.0
23330  53bf9f      1     0.0
23331  405a2b      1     0.0
23332  a40e01      1  3240.0

[23333 rows x 3 columns]
p-value равно: 0.199, H0 не можем отклонить


### Оценка среднего времени между покупками.

In [ ]:
# Оценить среднее время между покупками. Возьмите всех клиентов, у которых 2 и более покупок. 
# Вычислить время между покупками (для клиента с N покупками должно получиться N-1 значения времени). 
# Объединить значения всех клиентов и вычислите среднее.
# Файл 2022-04-01T12_df_sales.csv.


import os
from datetime import datetime
import pandas as pd
import numpy as np
from scipy import stats 

URL_BASE = 'https://raw.githubusercontent.com/ab-courses/simulator-ab-datasets/main/2022-04-01/'

def read_database(file_name):
    return pd.read_csv(os.path.join(URL_BASE, file_name))

df_sales = read_database('2022-04-01T12_df_sales.csv')
df_sales['date'] = pd.to_datetime(df_sales['date'])
df_sales.head()

,sale_id,date,count_pizza,count_drink,price,user_id
0,1000001,2022-02-04 10:00:24,1,0,720,1c1543
1,1000002,2022-02-04 10:02:28,1,1,930,a9a6e8
2,1000003,2022-02-04 10:02:35,3,1,1980,23420a
3,1000004,2022-02-04 10:03:06,1,1,750,3e8ed5
4,1000005,2022-02-04 10:03:23,1,1,870,cbc468


In [52]:
df_sales['count_sales'] = df_sales.groupby('user_id')['sale_id'].transform('count')
# print(df_sales.info())
df_sales_time_between = df_sales[df_sales['count_sales'] >=2].sort_values(['user_id', 'date'], ascending=True)
df_sales_time_between['prev_date'] = df_sales.groupby('user_id', as_index=False)['date'].shift(1)
df_sales_time_between['time_delta'] = (df_sales_time_between['date'] - df_sales_time_between['prev_date']) / pd.Timedelta(days=1)
mean_delta = round(df_sales_time_between['time_delta'].dropna().mean())
mean_delta

17

In [56]:
from pydantic import BaseModel

class Design(BaseModel):
    """Дата-класс с описанием параметров эксперимента.
    
    :param statistical_test: название статистического теста.
    :param metric: название метрики.
    :param alpha: уровень значимости.
    """
    statistical_test: str
    metric: str
    alpha: float

# Создание экземпляра класса с описанием конкретного эксперимента  
my_design = Design(
    statistical_test='t-test',
    metric='revenue',
    alpha=0.05
)

# Чтобы воспользоваться каким-либо значением из объекта данных, достаточно просто обратиться к нему через точку
print(my_design.alpha)

0.05


In [62]:
class User(BaseModel):
    login: str
    subscribed: bool = False
    age: int

user = User(login='winner2003', age='22', city='Moscow')
user.__dict__

{'login': 'winner2003', 'subscribed': False, 'age': 22}

#### ExperimentsService
Теперь мы начнём реализовывать центральный блок платформы A/B тестирования ExperimentsService, который отвечает за оценку экспериментов.

In [ ]:
# Теперь начнём реализовывать центральный блок платформы A/B тестирования ExperimentsService, который отвечает за оценку экспериментов.
# Метод get_pvalue класса ExperimentsService

import numpy as np
from pydantic import BaseModel
from scipy import stats


class Design(BaseModel):
    """Дата-класс с описание параметров эксперимента."""
    statistical_test: str


class ExperimentsService:

    def get_pvalue(self, metrics_a_group, metrics_b_group, design):
        """Применяет статтест, возвращает pvalue.

        :param metrics_a_group (np.array): массив значений метрик группы A
        :param metrics_a_group (np.array): массив значений метрик группы B
        :param design (Design): объект с данными, описывающий параметры эксперимента
        :return (float): значение p-value
        """
        if design.statistical_test == 'ttest':
            _, p_value = stats.ttest_ind(metrics_a_group, metrics_b_group)
            return p_value
        else:
            raise ValueError('Неверный design.statistical_test')

if __name__ == '__main__':
    metrics_a_group = np.array([964, 1123, 962, 1213, 914, 906, 951, 1033, 987, 1082])
    metrics_b_group = np.array([952, 1064, 1091, 1079, 1158, 921, 1161, 1064, 819, 1065])
    design = Design(statistical_test='ttest')
    ideal_pvalue = 0.612219

    experiments_service = ExperimentsService()
    pvalue = experiments_service.get_pvalue(metrics_a_group, metrics_b_group, design)
    np.testing.assert_almost_equal(ideal_pvalue, pvalue, decimal=4)
    print('simple test passed')
       

simple test passed
